# Cellular Automata

Finite state machines across space!

Rule 90: Discrete form of the logistic map: $x_{t+1} = r x_t (1 - x_t)$.

$$x_{i,t+1} = r (x_{i-1,t} + x_{i+1,t}) (2 - (x_{i-1,t} + x_{i+1,t})$$

If $r = 1$, simplifes to a set of rules:
 - If $x_{i-1,t} + x_{i+1,t} = 0$, then $x_{i,t+1} = 0$
 - If $x_{i-1,t} + x_{i+1,t} = 2$, then $x_{i,t+1} = 0$
 - If $x_{i-1,t} + x_{i+1,t} = 1$, then $x_{i,t+1} = 1$

In [ ]:
xx = matrix(0, 99, 100)
xx[50, 1] = 1

In [ ]:
image(xx)

In [ ]:
for (tt in 2:100) {
    for (ii in 2:98) {
        xx[ii, tt] = (xx[ii-1, tt-1] + xx[ii+1, tt-1])*(2 - (xx[ii-1, tt-1] + xx[ii+1, tt-1]))
    }
}

In [ ]:
image(xx)

# Sandpile model

Version 1: Add random grains

In [ ]:
cells = matrix(0, 40, 40)
for (ii in 1:1000) {
    rc = sample(40*40, 1)
    cells[rc] = cells[rc] + 1
}

In [ ]:
image(cells, zlim=c(0, 4))

In [ ]:
table(cells)

In [ ]:
which(cells >= 4)

In [ ]:
toppled = which(cells >= 4)
cells[toppled] = cells[toppled] - 4
cells[toppled - 1] = cells[toppled - 1] + 1
cells[toppled + 1] = cells[toppled + 1] + 1
cells[toppled - 40] = cells[toppled - 40] + 1
cells[toppled + 40] = cells[toppled + 40] + 1

In [ ]:
image(cells, zlim=c(0, 4))

Version 2: Include this in the loop

In [ ]:
size = 40
cells = matrix(sample(4, size*size, replace=T) - 1, size, size)
numtoppled = c()
for (ii in 1:10000) {
    rc = sample(size*size, 1)
    cells[rc] = cells[rc] + 1
    thistoppled = 0
    while (any(cells >= 4)) {
        toppled = which(cells >= 4)
        thistoppled = thistoppled + length(toppled)
        cells[toppled] = cells[toppled] - 4
        cells[pmax(1, toppled - 1)] = cells[pmax(1, toppled - 1)] + 1
        cells[pmin(size*size, toppled - 1)] = cells[pmin(size*size, toppled - 1)] + 1
        cells[pmax(1, toppled - 40)] = cells[pmax(1, toppled - 40)] + 1
        cells[pmin(size*size, toppled + 40)] = cells[pmin(size*size, toppled + 40)] + 1
        if (any(is.na(cells)))
            cancel
    }
    numtoppled = c(numtoppled, thistoppled)
}

In [ ]:
image(cells)

In [ ]:
sum(numtoppled)

In [ ]:
plot(numtoppled)

In [ ]:
library(ggplot2)
ggplot(data.frame(nn=numtoppled), aes(nn)) +
  geom_histogram()

In [ ]:
library(ggplot2)
ggplot(data.frame(nn=numtoppled), aes(nn)) +
  geom_histogram() + scale_x_log10() + scale_y_log10()

# Finding power law exponents

In [ ]:
install.packages("VGAM")
library(VGAM)

In [ ]:
ggplot(data.frame(x=rpareto(1e6, shape=1)), aes(x)) + geom_histogram() + scale_x_log10() + scale_y_log10()

In [ ]:
valeus = rpareto(1e6, shape=1)

In [ ]:
bins = seq(log(1), log(10000), length.out=100)

In [ ]:
count = c()
for (ii in 2:length(bins)) {
    count = c(count, sum(log(valeus) >= bins[ii-1] & log(valeus) < bins[ii]))
}

In [ ]:
logct = log(count)
logct[!is.finite(logct)] = NA
midbn = (bins[-1] + bins[-length(bins)]) / 2

In [ ]:
ggplot(data.frame(logct, midbn), aes(midbn, logct)) + geom_point() + geom_smooth(method='lm')

In [ ]:
summary(lm(logct ~ midbn))

## Fit by maximum likelihood

Power law distribution is:
$$p(x | \beta) = C x^{-\beta}$$

We have to know $C$, so calculate:
$$\int_{x_{min}}^\infty C x^{-\beta} dx = 1$$

Find $C = \frac{-\beta + 1}{x_{min}^{-\beta + 1}}$

Simplify the power law to:
$$p(x | \beta) = \frac{-\beta + 1}{x_{min}} \left(\frac{x}{x_{min}}\right)^{-\beta}$$

Solve for maximum likelihood of
$$\prod_i p(x_i | \beta)$$

Get $$\hat\beta = n \left(\sum_i \log{\left(\frac{x_i}{x_{min}}\right)}\right)^{-1}$$

In [ ]:
betahat = function(sizes, xmin) {
    length(sizes) / sum(log(sizes / xmin))
}

In [ ]:
betahat(valeus, 1)

In [ ]:
betahat(valeus[valeus > 10], 10)

# Looking at SOC data

In [ ]:
df = read.csv("soc-avalanche.csv")

In [ ]:
nrow(df)

In [ ]:
ggplot(df, aes(size)) + geom_histogram()

In [ ]:
ggplot(df, aes(size)) + geom_histogram() + scale_x_log10() + scale_y_log10()

In [ ]:
bins = seq(log(1), log(10000), length.out=100)

In [ ]:
count = c()
for (ii in 2:length(bins)) {
    count = c(count, sum(log(df$size) >= bins[ii-1] & log(df$size) < bins[ii]))
}

In [ ]:
plot(bins[-1], count, log='y')

In [ ]:
logct = log(count)
logct[!is.finite(logct)] = NA
midbn = (bins[-1] + bins[-length(bins)]) / 2

In [ ]:
summary(lm(logct ~ midbn))

In [ ]:
ggplot(data.frame(logct, midbn), aes(midbn, logct)) + geom_point() + geom_smooth(method='lm')

In [ ]:
exp(2.5)

In [ ]:
exp(8)

In [ ]:
betahat(df$size, 1)

In [ ]:
betahat(df$size[df$size < exp(2.5)], 1)

In [ ]:
betahat(df$size[df$size > exp(2.5) & df$size < exp(8)], exp(2.5))